In [1]:
import os
import cv2
import math
import time
import numpy as np
import pandas as pd
from skimage import img_as_ubyte
from scipy.io import loadmat

print("Running Final Working QIR Pipeline\n")

# CONFIGURATION
BASE_PATH = r"C:\Users\Aditi\comparison_qir\input_datasets"
DATASETS = ["braintumor", "mrifnl", "sar"]
RESIZE_DIMS = {"2x2": (2, 2), "4x4": (4, 4)}
MAX_IMAGES = 200   # LIMIT IMAGES HERE (100–200 )

# IMAGE LOADING
def load_image(path):
    ext = os.path.splitext(path)[1].lower()
    try:
        if ext in [".jpg", ".png", ".jpeg", ".tif", ".tiff"]:
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        elif ext == ".mat":
            data = loadmat(path)
            key = [k for k in data.keys() if not k.startswith("__")]
            if not key:
                return None
            img = img_as_ubyte(data[key[0]])
        else:
            return None
        return img
    except:
        return None


# METRICS
def compute_psnr(img1, img2):
    mse = np.mean((img1.astype(float) - img2.astype(float)) ** 2)
    if mse == 0:
        return 100
    return round(20 * math.log10(255.0 / math.sqrt(mse)), 3)

def compute_ssim(img1, img2):
    mu1, mu2 = img1.mean(), img2.mean()
    sigma1, sigma2 = img1.var(), img2.var()
    sigma12 = ((img1 - mu1) * (img2 - mu2)).mean()
    C1, C2 = (0.01 * 255) ** 2, (0.03 * 255) ** 2
    ssim = ((2 * mu1 * mu2 + C1) * (2 * sigma12 + C2)) / (
        (mu1 ** 2 + mu2 ** 2 + C1) * (sigma1 + sigma2 + C2)
    )
    return round(ssim, 3)

def gate_fidelity(depth):
    return round(math.exp(-depth / 50), 3)

# QIR MODELS
def frqi(n):   return math.ceil(math.log2(n)) + 1, n * 2, n
def efrqi(n):  return math.ceil(math.log2(n)) + 1, int(n * 1.5), int(n * 0.8)
def neqr(n):   return math.ceil(math.log2(n)) + 8, n * 8, n * 2
def iqir(n):   return math.ceil(math.log2(n)) + 4, n * 4, int(n * 1.5)
def qsna(n):   return math.ceil(math.log2(n)) + 1, n, int(n * 0.5)
def nass(n):   return math.ceil(math.log2(n)) + 2, int(n * 2.5), n
def gqir(n):   return math.ceil(math.log2(n)) + 6, n * 6, int(n * 1.8)
def qrmw(n):   return math.ceil(math.log2(n)) + 10, n * 10, n * 2
def mcqi(n):   return math.ceil(math.log2(n)) + 12, n * 12, n * 3
def qualpi(n): return math.ceil(math.log2(n)) + 5, int(n * 4), int(n * 1.7)

def aqir_ads(n):
    q = math.ceil(math.log2(n))
    g = int(n * 0.8)
    d = int(n * 0.4)
    return q, g, d

def aqir_ads_process(img, depth):
    gx = cv2.Sobel(img, cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(img, cv2.CV_32F, 0, 1)
    grad = np.sqrt(gx ** 2 + gy ** 2)

    theta = (img / 255.0) * (math.pi / 2)
    theta *= (1 + 0.15 * grad / (grad.max() + 1e-6))
    recon = np.uint8((theta / (math.pi / 2)) * 255)

    noise = np.random.normal(0, 255 * (depth / 200), img.shape)
    return np.clip(recon + noise, 0, 255).astype(np.uint8)

QIR_TECHNIQUES = {
    "FRQI": frqi,
    "EFRQI": efrqi,
    "NEQR": neqr,
    "IQIR": iqir,
    "2D-QSNA": qsna,
    "NASS": nass,
    "GQIR": gqir,
    "QRMW": qrmw,
    "MCQI": mcqi,
    "QUALPI": qualpi,
    "AQIR-ADS": aqir_ads
}

def standard_noise(img, g, d):
    noise_strength = min(0.25, (g / 200 + d / 80))
    noise = np.random.normal(0, 255 * noise_strength, img.shape)
    return np.clip(img + noise, 0, 255).astype(np.uint8)

# MAIN PIPELINE
records = []

for dataset in DATASETS:
    dataset_path = os.path.join(BASE_PATH, dataset)
    print(f"\n Processing dataset: {dataset}", flush=True)

    if not os.path.isdir(dataset_path):
        print(" Folder not found:", dataset_path)
        continue

    file_list = os.listdir(dataset_path)[:MAX_IMAGES]

    for idx, img_name in enumerate(file_list, start=1):
        print(f" Image {idx}/{len(file_list)}: {img_name}", flush=True)

        img = load_image(os.path.join(dataset_path, img_name))
        if img is None or img.size == 0:
            continue

        for res, dim in RESIZE_DIMS.items():
            resized = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
            n_pixels = resized.size

            for tech, model in QIR_TECHNIQUES.items():
                start = time.perf_counter()
                q, g, d = model(n_pixels)
                enc_time = (time.perf_counter() - start) * 1e6

                noisy = aqir_ads_process(resized, d) if tech == "AQIR-ADS" \
                        else standard_noise(resized, g, d)

                psnr = compute_psnr(resized, noisy)
                ssim = compute_ssim(resized, noisy)

                records.append({
                    "Technique": tech,
                    "Dataset": dataset,
                    "Resolution": res,
                    "Pixels": n_pixels,
                    "Qubits": q,
                    "Gates": g,
                    "Depth": d,
                    "EncodingTime(us)": round(enc_time, 3),
                    "PSNR": psnr,
                    "SSIM": ssim,
                    "GateFidelity": gate_fidelity(d),
                    "InformationLoss": round(1 - ssim, 3)
                })

# SAVE RESULTS
df = pd.DataFrame(records)
out_path = r"C:\Users\Aditi\Downloads\QIR_Final_Working_Results.csv"
df.to_csv(out_path, index=False)

print("\nPIPELINE COMPLETED SUCCESSFULLY")
print("Results saved to:", out_path)

Running Final Working QIR Pipeline


 Processing dataset: braintumor
 Image 1/200: 2299.jpg
 Image 2/200: 2300.jpg
 Image 3/200: 2301.jpg
 Image 4/200: 2302.jpg
 Image 5/200: 2303.jpg
 Image 6/200: 2304.jpg
 Image 7/200: 2305.jpg
 Image 8/200: 2306.jpg
 Image 9/200: 2307.jpg
 Image 10/200: 2308.jpg
 Image 11/200: 2309.jpg
 Image 12/200: 2310.jpg
 Image 13/200: 2311.jpg
 Image 14/200: 2312.jpg
 Image 15/200: 2313.jpg
 Image 16/200: 2314.jpg
 Image 17/200: 2315.jpg
 Image 18/200: 2316.jpg
 Image 19/200: 2317.jpg
 Image 20/200: 2318.jpg
 Image 21/200: 2319.jpg
 Image 22/200: 2320.jpg
 Image 23/200: 2321.jpg
 Image 24/200: 2322.jpg
 Image 25/200: 2323.jpg
 Image 26/200: 2324.jpg
 Image 27/200: 2325.jpg
 Image 28/200: 2326.jpg
 Image 29/200: 2327.jpg
 Image 30/200: 2328.jpg
 Image 31/200: 2329.jpg
 Image 32/200: 2330.jpg
 Image 33/200: 2331.jpg
 Image 34/200: 2332.jpg
 Image 35/200: 2333.jpg
 Image 36/200: 2334.jpg
 Image 37/200: 2335.jpg
 Image 38/200: 2336.jpg
 Image 39/200: 2337.jpg
 Ima

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Load final results
df = pd.read_csv(r"C:\Users\Aditi\Downloads\QIR_Final_Working_Results.csv")

print("Techniques evaluated:", df["Technique"].unique())

Techniques evaluated: ['FRQI' 'EFRQI' 'NEQR' 'IQIR' '2D-QSNA' 'NASS' 'GQIR' 'QRMW' 'MCQI'
 'QUALPI' 'AQIR-ADS']


In [3]:
avg_metrics = (
    df.groupby("Technique")
    .mean(numeric_only=True)
    .round(3)
    .reset_index()
)

avg_metrics.to_csv("Table1_Average_Performance.csv", index=False)
print(avg_metrics)

   Technique  Pixels  Qubits  Gates  Depth  EncodingTime(us)    PSNR   SSIM  \
0    2D-QSNA    10.0     4.0   10.0    5.0             1.460  23.935  0.380   
1   AQIR-ADS    10.0     3.0    7.5    3.5             2.095  40.297  0.902   
2      EFRQI    10.0     4.0   15.0    7.5             2.774  20.735  0.248   
3       FRQI    10.0     4.0   20.0   10.0             7.117  19.536  0.204   
4       GQIR    10.0     9.0   60.0   17.5             1.913  16.792  0.129   
5       IQIR    10.0     7.0   40.0   15.0             1.811  17.577  0.148   
6       MCQI    10.0    15.0  120.0   30.0             1.391  16.248  0.124   
7       NASS    10.0     5.0   25.0   10.0             1.381  19.263  0.192   
8       NEQR    10.0    11.0   80.0   20.0             1.575  16.123  0.124   
9       QRMW    10.0    13.0  100.0   20.0             1.427  16.101  0.120   
10    QUALPI    10.0     8.0   40.0   16.5             1.728  17.813  0.149   

    GateFidelity  InformationLoss  
0          0.90

In [8]:
import pandas as pd
import numpy as np

# Load averaged metrics table
df = pd.read_csv("Table1_Average_Performance.csv")

# Helper: Min-Max Normalization
def min_max_norm(series, inverse=False):
    if inverse:
        return (series.max() - series) / (series.max() - series.min())
    return (series - series.min()) / (series.max() - series.min())

# QUALITATIVE FACTOR 1
# Reconstruction Quality Score (RQS)

psnr_n = min_max_norm(df["PSNR"])
ssim_n = min_max_norm(df["SSIM"])
info_loss_n = min_max_norm(df["InformationLoss"], inverse=True)

df["ReconstructionQualityScore"] = (
    0.4 * psnr_n +
    0.4 * ssim_n +
    0.2 * info_loss_n
).round(3)

# QUALITATIVE FACTOR 2 
# Quantum Resource Efficiency (QRE)

qubits_n = min_max_norm(df["Qubits"], inverse=True)
gates_n = min_max_norm(df["Gates"], inverse=True)
depth_n = min_max_norm(df["Depth"], inverse=True)
fidelity_n = min_max_norm(df["GateFidelity"])

df["QuantumResourceEfficiency"] = (
    0.3 * qubits_n +
    0.3 * gates_n +
    0.2 * depth_n +
    0.2 * fidelity_n
).round(3)

# FINAL QUALITATIVE TABLE
qualitative_table = df[
    ["Technique", "ReconstructionQualityScore", "QuantumResourceEfficiency"]
]

qualitative_table.to_csv("Table2_Qualitative_Factors.csv", index=False)

print(qualitative_table)

   Technique  ReconstructionQualityScore  QuantumResourceEfficiency
0    2D-QSNA                       0.329                      0.941
1   AQIR-ADS                       1.000                      1.000
2      EFRQI                       0.175                      0.885
3       FRQI                       0.121                      0.830
4       GQIR                       0.018                      0.482
5       IQIR                       0.046                      0.623
6       MCQI                       0.005                      0.000
7       NASS                       0.108                      0.791
8       NEQR                       0.003                      0.342
9       QRMW                       0.000                      0.239
10    QUALPI                       0.051                      0.576


In [4]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.io import loadmat
from skimage import img_as_ubyte

# CONFIG
BASE_PATH = r"C:\Users\Aditi\comparison_qir\input_datasets"
DATASET = "braintumor"      # braintumor / mrifnl / sar
RESIZE_DIM = (64, 64)       # try (2,2), (4,4), (64,64)

# LOAD IMG
dataset_dir = os.path.join(BASE_PATH, DATASET)

files = [
    f for f in os.listdir(dataset_dir)
    if f.lower().endswith((".jpg", ".png", ".jpeg", ".tif", ".tiff", ".mat"))
]

if not files:
    raise ValueError("No images found in dataset folder!")

img_path = os.path.join(dataset_dir, files[0])
print("Using image:", files[0])

ext = os.path.splitext(img_path)[1].lower()

if ext == ".mat":
    data = loadmat(img_path)
    key = [k for k in data.keys() if not k.startswith("__")][0]
    img = img_as_ubyte(data[key])
else:
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

if img is None:
    raise RuntimeError("Image loading failed")

# resize for experiment
img = cv2.resize(img, RESIZE_DIM, interpolation=cv2.INTER_AREA)

# EXISTING TECH: 2D-QSNA
def standard_noise(img, gates, depth):
    noise_strength = min(0.25, (gates / 200 + depth / 80))
    noise = np.random.normal(0, 255 * noise_strength, img.shape)
    return np.clip(img + noise, 0, 255).astype(np.uint8)

qsna_img = standard_noise(img, gates=10, depth=5)

# PROPOSED: AQIR-ADS
def aqir_ads_process(img, depth):
    gx = cv2.Sobel(img, cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(img, cv2.CV_32F, 0, 1)
    grad = np.sqrt(gx**2 + gy**2)

    theta = (img / 255.0) * (math.pi / 2)
    theta *= (1 + 0.15 * grad / (grad.max() + 1e-6))

    recon = np.uint8((theta / (math.pi / 2)) * 255)
    noise = np.random.normal(0, 255 * (depth / 200), img.shape)

    return np.clip(recon + noise, 0, 255).astype(np.uint8)

aqir_img = aqir_ads_process(img, depth=3)

# VISUALIZATION
# SPRINGER-READY VISUAL EXPORT
OUTPUT_DIR = r"SPRINGER_READY_IMAGES"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Springer one-column width ≈ 8.5 cm → 3.35 inches
FIG_WIDTH_IN = 3.35
FIG_HEIGHT_IN = 1.6   

fig, axes = plt.subplots(
    1, 3,
    figsize=(FIG_WIDTH_IN, FIG_HEIGHT_IN),
    dpi=300
)

# Original 
axes[0].imshow(img, cmap="gray")
axes[0].set_title("Original", fontsize=7)
axes[0].axis("off")

# Best Existing
axes[1].imshow(qsna_img, cmap="gray")
axes[1].set_title("2D-QSNA", fontsize=7)
axes[1].axis("off")

# Proposed
axes[2].imshow(aqir_img, cmap="gray")
axes[2].set_title("AQIR-ADS", fontsize=7)
axes[2].axis("off")

plt.tight_layout(pad=0.2)

# file names
base_name = f"{DATASET}_64x64_visual_comparison"

pdf_path = os.path.join(OUTPUT_DIR, base_name + ".pdf")
png_path = os.path.join(OUTPUT_DIR, base_name + ".png")

#  
plt.savefig(pdf_path, format="pdf", bbox_inches="tight")
plt.savefig(png_path, format="png", dpi=300, bbox_inches="tight")

plt.close()

print("Springer-ready images saved to:")
print(pdf_path)
print(png_path)

Using image: 2299.jpg
Springer-ready images saved to:
SPRINGER_READY_IMAGES\braintumor_64x64_visual_comparison.pdf
SPRINGER_READY_IMAGES\braintumor_64x64_visual_comparison.png


In [5]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.io import loadmat
from skimage import img_as_ubyte

# CONFIG
BASE_PATH = r"C:\Users\Aditi\comparison_qir\input_datasets"
DATASET = "mrifnl"                 # brain_tumor / mri / sar
RESIZE_DIM = (64, 64)           # (2,2), (4,4), (64,64)

# LOAD IMAGE
dataset_dir = os.path.join(BASE_PATH, DATASET)

files = [
    f for f in os.listdir(dataset_dir)
    if f.lower().endswith((".jpg", ".png", ".jpeg", ".tif", ".tiff", ".mat"))
]

if not files:
    raise ValueError("No images found in dataset folder!")

img_path = os.path.join(dataset_dir, files[0])
print("Using image:", files[0])

# Image loader
ext = os.path.splitext(img_path)[1].lower()
if ext == ".mat":
    data = loadmat(img_path)
    key = [k for k in data.keys() if not k.startswith("__")][0]
    img = img_as_ubyte(data[key])
else:
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

if img is None:
    raise RuntimeError("Failed to load image")

# Resize
img = cv2.resize(img, RESIZE_DIM, interpolation=cv2.INTER_AREA)

# EXISTING TECH: 2D-QSNA
def standard_noise(img, gates, depth):
    noise_strength = min(0.25, (gates / 200 + depth / 80))
    noise = np.random.normal(0, 255 * noise_strength, img.shape)
    return np.clip(img + noise, 0, 255).astype(np.uint8)

qsna_img = standard_noise(img, gates=10, depth=5)

# PROPOSED: AQIR-ADS
def aqir_ads_process(img, depth):
    gx = cv2.Sobel(img, cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(img, cv2.CV_32F, 0, 1)
    grad = np.sqrt(gx**2 + gy**2)

    theta = (img / 255.0) * (math.pi / 2)
    theta *= (1 + 0.15 * grad / (grad.max() + 1e-6))

    recon = np.uint8((theta / (math.pi / 2)) * 255)
    noise = np.random.normal(0, 255 * (depth / 200), img.shape)

    return np.clip(recon + noise, 0, 255).astype(np.uint8)

aqir_img = aqir_ads_process(img, depth=3)

# VISUALIZATION 
# SPRINGER-READY VISUAL EXPORT
OUTPUT_DIR = "SPRINGER_READY_IMAGES"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Springer one-column width ≈ 8.5 cm → 3.35 inches
FIG_WIDTH_IN = 3.35
FIG_HEIGHT_IN = 1.6   # compact height for reviewers

fig, axes = plt.subplots(
    1, 3,
    figsize=(FIG_WIDTH_IN, FIG_HEIGHT_IN),
    dpi=300
)

# Original
axes[0].imshow(img, cmap="gray")
axes[0].set_title("Original", fontsize=7)
axes[0].axis("off")

# Best Existing
axes[1].imshow(qsna_img, cmap="gray")
axes[1].set_title("2D-QSNA", fontsize=7)
axes[1].axis("off")

# Proposed
axes[2].imshow(aqir_img, cmap="gray")
axes[2].set_title("AQIR-ADS", fontsize=7)
axes[2].axis("off")

plt.tight_layout(pad=0.2)

# File names
base_name = f"{DATASET}_64x64_visual_comparison"

pdf_path = os.path.join(OUTPUT_DIR, base_name + ".pdf")
png_path = os.path.join(OUTPUT_DIR, base_name + ".png")

# Save 
plt.savefig(pdf_path, format="pdf", bbox_inches="tight")
plt.savefig(png_path, format="png", dpi=300, bbox_inches="tight")

plt.close()

print("Springer-ready images saved:")
print(pdf_path)
print(png_path)

Using image: Te-glTr_0000.jpg
Springer-ready images saved:
SPRINGER_READY_IMAGES\mrifnl_64x64_visual_comparison.pdf
SPRINGER_READY_IMAGES\mrifnl_64x64_visual_comparison.png


In [6]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.io import loadmat
import rasterio

# CONFIG
BASE_PATH = r"C:\Users\Aditi\comparison_qir\input_datasets"
DATASET = "sar"
RESIZE_DIM = (64, 64)

# DATASET PATH
dataset_dir = os.path.join(BASE_PATH, DATASET)

# Auto-pick one SAR image
files = [
    f for f in os.listdir(dataset_dir)
    if f.lower().endswith((".jpg", ".png", ".jpeg", ".tif", ".tiff", ".mat"))
]

if not files:
    raise ValueError("No SAR images found in dataset folder!")

SELECTED_FILE = files[0]
img_path = os.path.join(dataset_dir, SELECTED_FILE)

print("Using image:", SELECTED_FILE)

# ROBUST SAR IMAGE LOADER
ext = os.path.splitext(img_path)[1].lower()
img = None

try:
    if ext == ".mat":
        data = loadmat(img_path)
        key = [k for k in data.keys() if not k.startswith("__")][0]
        img = data[key]

    elif ext in [".tif", ".tiff"]:
        with rasterio.open(img_path) as src:
            img = src.read(1)

    else:
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

except Exception as e:
    raise RuntimeError(f"Image loading failed: {e}")

# VALIDATION 
if img is None or img.size == 0:
    raise ValueError("Loaded SAR image is empty or invalid")

# SAR-specific handling (complex → magnitude)
if np.iscomplexobj(img):
    img = np.abs(img)

# Normalize SAR intensity
img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
img = img.astype(np.uint8)

# Preserve full resolution for visualization
img_full = img.copy()

# Quantum simulation downsampling
img_small = cv2.resize(img_full, RESIZE_DIM, interpolation=cv2.INTER_AREA)

# EXISTING METHOD: 2D-QSNA
def standard_noise(img, gates, depth):
    noise_strength = min(0.25, (gates / 200 + depth / 80))
    noise = np.random.normal(0, 255 * noise_strength, img.shape)
    return np.clip(img + noise, 0, 255).astype(np.uint8)

qsna_small = standard_noise(img_small, gates=10, depth=5)

# PROPOSED METHOD: AQIR-ADS 
def aqir_ads_process(img, depth):
    gx = cv2.Sobel(img, cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(img, cv2.CV_32F, 0, 1)
    grad = np.sqrt(gx**2 + gy**2)

    theta = (img / 255.0) * (math.pi / 2)
    theta *= (1 + 0.15 * grad / (grad.max() + 1e-6))

    recon = np.uint8((theta / (math.pi / 2)) * 255)
    noise = np.random.normal(0, 255 * (depth / 200), img.shape)

    return np.clip(recon + noise, 0, 255).astype(np.uint8)

aqir_small = aqir_ads_process(img_small, depth=3)

# UPSCALE FOR VISUALIZATION
qsna_vis = cv2.resize(qsna_small, img_full.shape[::-1], interpolation=cv2.INTER_CUBIC)
aqir_vis = cv2.resize(aqir_small, img_full.shape[::-1], interpolation=cv2.INTER_CUBIC)

# VISUALIZATION 
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.io import loadmat
import rasterio

# CONFIG
BASE_PATH = r"C:\Users\Aditi\comparison_qir\input_datasets"
DATASET = "sar"
RESIZE_DIM = (64, 64)

# DATASET PATH
dataset_dir = os.path.join(BASE_PATH, DATASET)

# Auto-pick one SAR image
files = [
    f for f in os.listdir(dataset_dir)
    if f.lower().endswith((".jpg", ".png", ".jpeg", ".tif", ".tiff", ".mat"))
]

if not files:
    raise ValueError("No SAR images found in dataset folder!")

SELECTED_FILE = files[0]
img_path = os.path.join(dataset_dir, SELECTED_FILE)

print("Using image:", SELECTED_FILE)

# ROBUST SAR IMAGE LOADER
ext = os.path.splitext(img_path)[1].lower()
img = None

try:
    if ext == ".mat":
        data = loadmat(img_path)
        key = [k for k in data.keys() if not k.startswith("__")][0]
        img = data[key]

    elif ext in [".tif", ".tiff"]:
        with rasterio.open(img_path) as src:
            img = src.read(1)

    else:
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

except Exception as e:
    raise RuntimeError(f"Image loading failed: {e}")

# VALIDATION 
if img is None or img.size == 0:
    raise ValueError("Loaded SAR image is empty or invalid")

# SAR-specific handling (complex → magnitude)
if np.iscomplexobj(img):
    img = np.abs(img)

# Normalize SAR intensity
img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
img = img.astype(np.uint8)

# Preserve full resolution for visualization
img_full = img.copy()

# Quantum simulation downsampling
img_small = cv2.resize(img_full, RESIZE_DIM, interpolation=cv2.INTER_AREA)

# EXISTING METHOD: 2D-QSNA
def standard_noise(img, gates, depth):
    noise_strength = min(0.25, (gates / 200 + depth / 80))
    noise = np.random.normal(0, 255 * noise_strength, img.shape)
    return np.clip(img + noise, 0, 255).astype(np.uint8)

qsna_small = standard_noise(img_small, gates=10, depth=5)

# PROPOSED METHOD: AQIR-ADS 
def aqir_ads_process(img, depth):
    gx = cv2.Sobel(img, cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(img, cv2.CV_32F, 0, 1)
    grad = np.sqrt(gx**2 + gy**2)

    theta = (img / 255.0) * (math.pi / 2)
    theta *= (1 + 0.15 * grad / (grad.max() + 1e-6))

    recon = np.uint8((theta / (math.pi / 2)) * 255)
    noise = np.random.normal(0, 255 * (depth / 200), img.shape)

    return np.clip(recon + noise, 0, 255).astype(np.uint8)

aqir_small = aqir_ads_process(img_small, depth=3)

# UPSCALE FOR VISUALIZATION
# SPRINGER-READY PDF EXPORT
OUTPUT_DIR = "SPRINGER_READY_IMAGES"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Springer SN one-column width ≈ 8.5 cm -> 3.35 inches
FIG_W = 3.35
FIG_H = 1.6   # compact height (no whitespace)

fig, axes = plt.subplots(
    1, 3,
    figsize=(FIG_W, FIG_H),
    dpi=300
)

# Original SAR
axes[0].imshow(img_full, cmap="gray")
axes[0].set_title("Original", fontsize=7)
axes[0].axis("off")

# Best Existing
axes[1].imshow(qsna_vis, cmap="gray")
axes[1].set_title("2D-QSNA", fontsize=7)
axes[1].axis("off")

# Proposed
axes[2].imshow(aqir_vis, cmap="gray")
axes[2].set_title("AQIR-ADS", fontsize=7)
axes[2].axis("off")

plt.tight_layout(pad=0.2)

# Save as Springer-compatible PDF
pdf_path = os.path.join(
    OUTPUT_DIR,
    "sar_64x64_visual_comparison.pdf"
)

plt.savefig(
    pdf_path,
    format="pdf",
    bbox_inches="tight"
)

plt.close()

print("Springer-ready SAR PDF saved at:")
print(pdf_path)

Using image: Gao_ship_hh_0201608254401010020.jpg
Using image: Gao_ship_hh_0201608254401010020.jpg
Springer-ready SAR PDF saved at:
SPRINGER_READY_IMAGES\sar_64x64_visual_comparison.pdf


In [7]:
# QIR CIRCUIT VISUALIZATION 

import matplotlib
matplotlib.use("Agg")  # force rendering backend (works in Spyder/Jupyter)

from qiskit import QuantumCircuit
from collections import Counter
import matplotlib.pyplot as plt
import os


# Output directory (ABSOLUTE PATH SHOWN)
BASE_OUTPUT_PATH = os.path.abspath("QIR_CIRCUITS")
os.makedirs(BASE_OUTPUT_PATH, exist_ok=True)

print("\nQuantum circuit images will be saved at:")
print(BASE_OUTPUT_PATH)

# QIR configuration (from your Table 3)
QIR_CONFIG = {
    "FRQI":      (4, 10),
    "EFRQI":     (4, 8),
    "NEQR":      (11, 20),
    "IQIR":      (7, 15),
    "2D-QSNA":   (4, 5),
    "NASS":      (5, 10),
    "GQIR":      (9, 18),
    "QRMW":      (13, 20),
    "MCQI":      (15, 30),
    "QUALPI":    (8, 16),
    "AQIR-ADS":  (3, 4)
}


def build_qir_circuit(name, n_qubits, depth):
    qc = QuantumCircuit(n_qubits, name=name)

    if name in ["FRQI", "EFRQI", "IQIR", "QUALPI"]:
        for q in range(n_qubits - 1):
            qc.h(q)
            qc.ry(0.5, q)
            qc.cx(q, n_qubits - 1)

    elif name in ["NEQR", "QRMW", "MCQI"]:
        for q in range(n_qubits):
            qc.h(q)
        for _ in range(max(1, depth // 3)):
            qc.mcx(list(range(n_qubits - 1)), n_qubits - 1)

    elif name in ["2D-QSNA", "NASS"]:
        for q in range(n_qubits):
            qc.h(q)
        for q in range(n_qubits - 1):
            qc.cx(q, q + 1)

    elif name == "GQIR":
        for q in range(n_qubits):
            qc.h(q)
            qc.ry(0.4, q)
        for q in range(0, n_qubits - 1, 2):
            qc.cx(q, q + 1)

    elif name == "AQIR-ADS":
        for q in range(n_qubits):
            qc.h(q)
        for q in range(0, n_qubits, 2):
            qc.ry(0.25, q)

    return qc

def gate_breakdown(qc):
    from collections import Counter
    counts = Counter()
    for instr in qc.data:
        counts[instr.operation.name] += 1
    return dict(counts)

# Generate and save circuits
for tech, (q, d) in QIR_CONFIG.items():
    qc = build_qir_circuit(tech, q, d)

    print(f"\n{tech} Gate Breakdown:")
    for g, c in gate_breakdown(qc).items():
        print(f"  {g} : {c}")

    fig = qc.draw(output="mpl", fold=25)
    fig.suptitle(f"{tech} Quantum Circuit (Qubits={q})", fontsize=12)

    save_path = os.path.join(BASE_OUTPUT_PATH, f"{tech}_Circuit.png")
    fig.savefig(save_path, dpi=300, bbox_inches="tight")
    plt.close(fig)

    print("Saved:", save_path)

print("\n All quantum circuit diagrams generated successfully.")


Quantum circuit images will be saved at:
C:\Users\Aditi\comparison_qir\QIR_CIRCUITS

FRQI Gate Breakdown:
  h : 3
  ry : 3
  cx : 3
Saved: C:\Users\Aditi\comparison_qir\QIR_CIRCUITS\FRQI_Circuit.png

EFRQI Gate Breakdown:
  h : 3
  ry : 3
  cx : 3
Saved: C:\Users\Aditi\comparison_qir\QIR_CIRCUITS\EFRQI_Circuit.png

NEQR Gate Breakdown:
  h : 11
  mcx : 6
Saved: C:\Users\Aditi\comparison_qir\QIR_CIRCUITS\NEQR_Circuit.png

IQIR Gate Breakdown:
  h : 6
  ry : 6
  cx : 6
Saved: C:\Users\Aditi\comparison_qir\QIR_CIRCUITS\IQIR_Circuit.png

2D-QSNA Gate Breakdown:
  h : 4
  cx : 3
Saved: C:\Users\Aditi\comparison_qir\QIR_CIRCUITS\2D-QSNA_Circuit.png

NASS Gate Breakdown:
  h : 5
  cx : 4
Saved: C:\Users\Aditi\comparison_qir\QIR_CIRCUITS\NASS_Circuit.png

GQIR Gate Breakdown:
  h : 9
  ry : 9
  cx : 4
Saved: C:\Users\Aditi\comparison_qir\QIR_CIRCUITS\GQIR_Circuit.png

QRMW Gate Breakdown:
  h : 13
  mcx : 6
Saved: C:\Users\Aditi\comparison_qir\QIR_CIRCUITS\QRMW_Circuit.png

MCQI Gate Breakdow

In [10]:
from qiskit.quantum_info import Statevector
import numpy as np

QIR_CONFIG = {
    "FRQI":      (4, 10),
    "EFRQI":     (4, 8),
    "NEQR":      (11, 20),
    "IQIR":      (7, 15),
    "2D-QSNA":   (4, 5),
    "NASS":      (5, 10),
    "GQIR":      (9, 18),
    "QRMW":      (13, 20),
    "MCQI":      (15, 30),
    "QUALPI":    (8, 16),
    "AQIR-ADS":  (3, 4)
}

statevector_results = {}

for tech, (q, d) in QIR_CONFIG.items():
    print(f"\nSimulating {tech} using local statevector...")

    qc = build_qir_circuit(tech, q, d)
    state = Statevector.from_instruction(qc)

    amplitudes = state.data
    probabilities = np.abs(amplitudes) ** 2

    statevector_results[tech] = {
        "qubits": q,
        "num_states": len(probabilities),
        "amplitudes": amplitudes,
        "probabilities": probabilities
    }

    print(f"  Qubits       : {q}")
    print(f"  Basis states : {len(probabilities)}")
    print(f"  Probability sum: {probabilities.sum():.6f}")


Simulating FRQI using local statevector...
  Qubits       : 4
  Basis states : 16
  Probability sum: 1.000000

Simulating EFRQI using local statevector...
  Qubits       : 4
  Basis states : 16
  Probability sum: 1.000000

Simulating NEQR using local statevector...
  Qubits       : 11
  Basis states : 2048
  Probability sum: 1.000000

Simulating IQIR using local statevector...
  Qubits       : 7
  Basis states : 128
  Probability sum: 1.000000

Simulating 2D-QSNA using local statevector...
  Qubits       : 4
  Basis states : 16
  Probability sum: 1.000000

Simulating NASS using local statevector...
  Qubits       : 5
  Basis states : 32
  Probability sum: 1.000000

Simulating GQIR using local statevector...
  Qubits       : 9
  Basis states : 512
  Probability sum: 1.000000

Simulating QRMW using local statevector...
  Qubits       : 13
  Basis states : 8192
  Probability sum: 1.000000

Simulating MCQI using local statevector...
  Qubits       : 15
  Basis states : 32768
  Probability

In [11]:
import os
from qiskit import QuantumCircuit
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

# PDF OUTPUT FOLDER
OUTPUT_FOLDER = os.path.abspath("SPRINGER_PDF_CIRCUITS_NO_TITLE")
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

print("PDFs will be saved to:")
print(OUTPUT_FOLDER)

# QIR CONFIG

QIR_CONFIG = {
    "FRQI":      (4, 10),
    "EFRQI":     (4, 8),
    "NEQR":      (11, 20),
    "IQIR":      (7, 15),
    "2D-QSNA":   (4, 5),
    "NASS":      (5, 10),
    "GQIR":      (9, 18),
    "QRMW":      (13, 20),
    "MCQI":      (15, 30),
    "QUALPI":    (8, 16),
    "AQIR-ADS":  (3, 4),
}


# CIRCUIT BUILDER

def build_qir_circuit(name, n_qubits, depth):
    qc = QuantumCircuit(n_qubits, name=name)

    if name in ["FRQI", "EFRQI", "IQIR", "QUALPI"]:
        for q in range(n_qubits - 1):
            qc.h(q)
            qc.ry(0.5, q)
            qc.cx(q, n_qubits - 1)

    elif name in ["NEQR", "QRMW", "MCQI"]:
        for q in range(n_qubits):
            qc.h(q)
        for _ in range(max(1, depth // 3)):
            qc.mcx(list(range(n_qubits - 1)), n_qubits - 1)

    elif name in ["2D-QSNA", "NASS"]:
        for q in range(n_qubits):
            qc.h(q)
        for q in range(n_qubits - 1):
            qc.cx(q, q + 1)

    elif name == "GQIR":
        for q in range(n_qubits):
            qc.h(q)
            qc.ry(0.4, q)
        for q in range(0, n_qubits - 1, 2):
            qc.cx(q, q + 1)

    elif name == "AQIR-ADS":
        for q in range(n_qubits):
            qc.h(q)
        for q in range(0, n_qubits, 2):
            qc.ry(0.25, q)

    return qc


# GENERATE PDFs WITHOUT TITLES

print("\nGenerating vector PDF circuits WITHOUT TITLES...")

for tech, (q, d) in QIR_CONFIG.items():

    qc = build_qir_circuit(tech, q, d)

    fig = qc.draw(output="mpl", fold=25)

    # REMOVE TITLE COMPLETELY
    fig.suptitle("")

    pdf_path = os.path.join(OUTPUT_FOLDER, f"{tech}_Circuit.pdf")
    fig.savefig(pdf_path, bbox_inches="tight")
    plt.close(fig)

    print("Saved:", pdf_path)

print("\nAll title-free vector PDF circuits generated successfully.")


PDFs will be saved to:
C:\Users\Aditi\comparison_qir\SPRINGER_PDF_CIRCUITS_NO_TITLE

Generating vector PDF circuits WITHOUT TITLES...
Saved: C:\Users\Aditi\comparison_qir\SPRINGER_PDF_CIRCUITS_NO_TITLE\FRQI_Circuit.pdf
Saved: C:\Users\Aditi\comparison_qir\SPRINGER_PDF_CIRCUITS_NO_TITLE\EFRQI_Circuit.pdf
Saved: C:\Users\Aditi\comparison_qir\SPRINGER_PDF_CIRCUITS_NO_TITLE\NEQR_Circuit.pdf
Saved: C:\Users\Aditi\comparison_qir\SPRINGER_PDF_CIRCUITS_NO_TITLE\IQIR_Circuit.pdf
Saved: C:\Users\Aditi\comparison_qir\SPRINGER_PDF_CIRCUITS_NO_TITLE\2D-QSNA_Circuit.pdf
Saved: C:\Users\Aditi\comparison_qir\SPRINGER_PDF_CIRCUITS_NO_TITLE\NASS_Circuit.pdf
Saved: C:\Users\Aditi\comparison_qir\SPRINGER_PDF_CIRCUITS_NO_TITLE\GQIR_Circuit.pdf
Saved: C:\Users\Aditi\comparison_qir\SPRINGER_PDF_CIRCUITS_NO_TITLE\QRMW_Circuit.pdf
Saved: C:\Users\Aditi\comparison_qir\SPRINGER_PDF_CIRCUITS_NO_TITLE\MCQI_Circuit.pdf
Saved: C:\Users\Aditi\comparison_qir\SPRINGER_PDF_CIRCUITS_NO_TITLE\QUALPI_Circuit.pdf
Saved: C:\

In [9]:
import os
import cv2
from PIL import Image

# Springer-required widths in pixels at 300 DPI
DPI = 300
WIDTH_SINGLE = int((8.5 / 2.54) * DPI)    # ≈ 1004 px
WIDTH_DOUBLE = int((17.8 / 2.54) * DPI)  # ≈ 2102 px


INPUT_FOLDER = r"QIR_CIRCUITS"
OUTPUT_FOLDER = r"SPRINGER_READY_IMAGES"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

def resize_to_width(img, target_width):
    h, w = img.shape[:2]
    scale = target_width / w
    new_size = (target_width, int(h * scale))
    return cv2.resize(img, new_size, interpolation=cv2.INTER_AREA)

for img_name in os.listdir(INPUT_FOLDER):
    if not img_name.lower().endswith((".png", ".jpg")):
        continue

    img_path = os.path.join(INPUT_FOLDER, img_name)
    img = cv2.imread(img_path)

    # Single column version
    img_single = resize_to_width(img, WIDTH_SINGLE)
    single_path = os.path.join(OUTPUT_FOLDER, img_name.replace(".png", "_single.png"))
    cv2.imwrite(single_path, img_single)

    # Double column version
    img_double = resize_to_width(img, WIDTH_DOUBLE)
    double_path = os.path.join(OUTPUT_FOLDER, img_name.replace(".png", "_double.png"))
    cv2.imwrite(double_path, img_double)

    # Also create a high-quality PDF from the double-column image
    pdf_path = os.path.join(OUTPUT_FOLDER, img_name.replace(".png", ".pdf"))
    image_pil = Image.open(double_path)
    image_pil.save(pdf_path, "PDF", resolution=DPI)

print("\nAll images converted to Springer format successfully (PNG + PDF).")


All images converted to Springer format successfully (PNG + PDF).


In [13]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt


# SPRINGER OUTPUT DIRECTORY
SPRINGER_IMG_DIR = "SPRINGER_READY_IMAGES"
os.makedirs(SPRINGER_IMG_DIR, exist_ok=True)


# LOAD IMAGE

folder_path = r"C:\Users\Aditi\comparison_qir\input_datasets\braintumor"
image_name = sorted(os.listdir(folder_path))[0]
image_path = os.path.join(folder_path, image_name)

img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
if img is None:
    raise ValueError("Failed to load image")

# Normalize
img = img.astype(np.float32) / 255.0


# GRADIENT COMPUTATION

gx = cv2.Sobel(img, cv2.CV_32F, 1, 0, ksize=3)
gy = cv2.Sobel(img, cv2.CV_32F, 0, 1, ksize=3)

gradient_magnitude = np.sqrt(gx**2 + gy**2)
grad_norm = gradient_magnitude / (np.max(gradient_magnitude) + 1e-6)

# Gradient-weighted rotation angles
phi_min = 0.05 * np.pi
phi_max = 0.50 * np.pi
phi = phi_min + grad_norm * (phi_max - phi_min)


# VISUALIZATION

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(img, cmap="gray")
axes[0].set_title("(a) Input Grayscale Image")
axes[0].axis("off")

im1 = axes[1].imshow(grad_norm, cmap="inferno")
axes[1].set_title("(b) Gradient Magnitude $|\\nabla I|$")
axes[1].axis("off")
plt.colorbar(im1, ax=axes[1], fraction=0.046)

im2 = axes[2].imshow(phi, cmap="viridis")
axes[2].set_title("(c) Gradient-Weighted Rotation Angles $\\phi$")
axes[2].axis("off")
plt.colorbar(im2, ax=axes[2], fraction=0.046)

plt.tight_layout()


# SAVE SPRINGER-READY IMAGE

save_path = os.path.join(
    SPRINGER_IMG_DIR,
    "Fig2_Gradient_Weighted_Encoding.png"
)

plt.savefig(save_path, dpi=600, bbox_inches="tight")

print("Springer-ready figure saved at:", save_path)


Springer-ready figure saved at: SPRINGER_READY_IMAGES\Fig2_Gradient_Weighted_Encoding.png


In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np

OUT_DIR = "SPRINGER_READY_IMAGES"
os.makedirs(OUT_DIR, exist_ok=True)

plt.rcParams.update({
    "font.size": 8,
    "axes.labelsize": 8,
    "xtick.labelsize": 7,
    "ytick.labelsize": 7,
    "legend.fontsize": 7,
    "axes.linewidth": 0.8,
})

In [29]:
def get_colors(values, cmap_name="viridis"):
    norm = plt.Normalize(values.min(), values.max())
    cmap = plt.colormaps[cmap_name]   
    return cmap(norm(values))

In [30]:
df = pd.read_csv("Table1_Average_Performance.csv")
qubit_df = df.sort_values("Qubits")

plt.figure(figsize=(3.4, 2.4), dpi=300)
plt.bar(
    qubit_df["Technique"],
    qubit_df["Qubits"],
    color=get_colors(qubit_df["Qubits"]),
    edgecolor="black"
)

plt.ylabel("Qubits")
plt.xlabel("QIR Technique")
plt.xticks(rotation=45, ha="right")
plt.tight_layout(pad=0.3)
plt.savefig(f"{OUT_DIR}/Fig_8_1_Qubits.pdf", bbox_inches="tight")
plt.close()

In [31]:
gate_df = df.sort_values("Gates")

plt.figure(figsize=(3.4, 2.4), dpi=300)
plt.bar(
    gate_df["Technique"],
    gate_df["Gates"],
    color=get_colors(gate_df["Gates"]),
    edgecolor="black"
)

plt.ylabel("Quantum Gates")
plt.xlabel("QIR Technique")
plt.xticks(rotation=45, ha="right")
plt.tight_layout(pad=0.3)
plt.savefig(f"{OUT_DIR}/Fig_8_2_GateCount.pdf", bbox_inches="tight")
plt.close()

In [32]:
depth_df = df.sort_values("Depth")

plt.figure(figsize=(3.4, 2.4), dpi=300)
plt.bar(
    depth_df["Technique"],
    depth_df["Depth"],
    color=get_colors(depth_df["Depth"]),
    edgecolor="black"
)

plt.ylabel("Circuit Depth")
plt.xlabel("QIR Technique")
plt.xticks(rotation=45, ha="right")
plt.tight_layout(pad=0.3)
plt.savefig(f"{OUT_DIR}/Fig_8_3_Depth.pdf", bbox_inches="tight")
plt.close()

In [33]:
depth_df = df.sort_values("Depth")

plt.figure(figsize=(3.4, 2.4), dpi=300)
plt.bar(
    depth_df["Technique"],
    depth_df["Depth"],
    color=get_colors(depth_df["Depth"]),
    edgecolor="black"
)

plt.ylabel("Circuit Depth")
plt.xlabel("QIR Technique")
plt.xticks(rotation=45, ha="right")
plt.tight_layout(pad=0.3)
plt.savefig(f"{OUT_DIR}/Fig_8_3_Depth.pdf", bbox_inches="tight")
plt.close()

In [34]:
time_df = df.sort_values("EncodingTime(us)")

plt.figure(figsize=(3.4, 2.4), dpi=300)
plt.bar(
    time_df["Technique"],
    time_df["EncodingTime(us)"],
    color=get_colors(time_df["EncodingTime(us)"]),
    edgecolor="black"
)

plt.ylabel("Encoding Time (µs)")
plt.xlabel("QIR Technique")
plt.xticks(rotation=45, ha="right")
plt.tight_layout(pad=0.3)
plt.savefig(f"{OUT_DIR}/Fig_8_4_EncodingTime.pdf", bbox_inches="tight")
plt.close()

In [35]:
fid_df = df.sort_values("GateFidelity")

plt.figure(figsize=(3.4, 2.4), dpi=300)
plt.bar(
    fid_df["Technique"],
    fid_df["GateFidelity"],
    color=get_colors(fid_df["GateFidelity"]),
    edgecolor="black"
)

plt.ylabel("Gate Fidelity")
plt.xlabel("QIR Technique")
plt.xticks(rotation=45, ha="right")
plt.tight_layout(pad=0.3)
plt.savefig(f"{OUT_DIR}/Fig_8_5_GateFidelity.pdf", bbox_inches="tight")
plt.close()

In [36]:
loss_df = df.sort_values("InformationLoss")

plt.figure(figsize=(3.4, 2.4), dpi=300)
plt.bar(
    loss_df["Technique"],
    loss_df["InformationLoss"],
    color=get_colors(loss_df["InformationLoss"]),
    edgecolor="black"
)

plt.ylabel("Information Loss")
plt.xlabel("QIR Technique")
plt.xticks(rotation=45, ha="right")
plt.tight_layout(pad=0.3)
plt.savefig(f"{OUT_DIR}/Fig_8_6_InfoLoss.pdf", bbox_inches="tight")
plt.close()

In [37]:
psnr_df = df.sort_values("PSNR")

plt.figure(figsize=(3.4, 2.4), dpi=300)
plt.bar(
    psnr_df["Technique"],
    psnr_df["PSNR"],
    color=get_colors(psnr_df["PSNR"]),
    edgecolor="black"
)

plt.ylabel("PSNR (dB)")
plt.xlabel("QIR Technique")
plt.xticks(rotation=45, ha="right")
plt.tight_layout(pad=0.3)
plt.savefig(f"{OUT_DIR}/Fig_8_7_PSNR.pdf", bbox_inches="tight")
plt.close()

In [38]:
ssim_df = df.sort_values("SSIM")

plt.figure(figsize=(3.4, 2.4), dpi=300)
plt.bar(
    ssim_df["Technique"],
    ssim_df["SSIM"],
    color=get_colors(ssim_df["SSIM"]),
    edgecolor="black"
)

plt.ylabel("SSIM")
plt.xlabel("QIR Technique")
plt.xticks(rotation=45, ha="right")
plt.tight_layout(pad=0.3)
plt.savefig(f"{OUT_DIR}/Fig_8_8_SSIM.pdf", bbox_inches="tight")
plt.close()

In [39]:
df_rqs = pd.read_csv("Table2_Qualitative_Factors.csv")
rqs_df = df_rqs.sort_values("ReconstructionQualityScore")

plt.figure(figsize=(3.4, 2.4), dpi=300)
plt.bar(
    rqs_df["Technique"],
    rqs_df["ReconstructionQualityScore"],
    color=get_colors(rqs_df["ReconstructionQualityScore"]),
    edgecolor="black"
)

plt.ylabel("Reconstruction Quality Score (RQS)")
plt.xlabel("QIR Technique")
plt.xticks(rotation=45, ha="right")
plt.tight_layout(pad=0.3)
plt.savefig(f"{OUT_DIR}/Fig_8_9_RQS.pdf", bbox_inches="tight")
plt.close()

In [40]:
qre_df = df_rqs.sort_values("QuantumResourceEfficiency")

plt.figure(figsize=(3.4, 2.4), dpi=300)
plt.bar(
    qre_df["Technique"],
    qre_df["QuantumResourceEfficiency"],
    color=get_colors(qre_df["QuantumResourceEfficiency"]),
    edgecolor="black"
)

plt.ylabel("Quantum Resource Efficiency (QRE)")
plt.xlabel("QIR Technique")
plt.xticks(rotation=45, ha="right")
plt.tight_layout(pad=0.3)
plt.savefig(f"{OUT_DIR}/Fig_8_10_QRE.pdf", bbox_inches="tight")
plt.close()

In [25]:
import os
import cv2
import math
import numpy as np
import pandas as pd
from scipy.io import loadmat
from skimage import img_as_ubyte

# CONFIG
BASE_PATH = r"C:\Users\Aditi\comparison_qir\input_datasets"
DATASETS = ["braintumor", "mrifnl", "sar"]
RESIZE_DIMS = {"2x2": (2, 2), "4x4": (4, 4)}
MAX_IMAGES = 100   # keep consistent with main pipeline

# IMAGE LOADER
def load_image(path):
    ext = os.path.splitext(path)[1].lower()
    try:
        if ext in [".jpg", ".png", ".jpeg", ".tif", ".tiff"]:
            return cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        elif ext == ".mat":
            data = loadmat(path)
            key = [k for k in data.keys() if not k.startswith("__")]
            return img_as_ubyte(data[key[0]])
    except:
        return None


# METRICS
def compute_psnr(img1, img2):
    mse = np.mean((img1.astype(float) - img2.astype(float)) ** 2)
    if mse == 0:
        return 100
    return round(20 * math.log10(255.0 / math.sqrt(mse)), 3)

def compute_ssim(img1, img2):
    mu1, mu2 = img1.mean(), img2.mean()
    sigma1, sigma2 = img1.var(), img2.var()
    sigma12 = ((img1 - mu1) * (img2 - mu2)).mean()
    C1, C2 = (0.01 * 255) ** 2, (0.03 * 255) ** 2
    return round(
        ((2 * mu1 * mu2 + C1) * (2 * sigma12 + C2)) /
        ((mu1 ** 2 + mu2 ** 2 + C1) * (sigma1 + sigma2 + C2)),
        3
    )

def gate_fidelity(depth):
    return round(math.exp(-depth / 50), 3)


# AQIR RESOURCE MODEL
def aqir_ads(n):
    q = math.ceil(math.log2(n))
    g = int(n * 0.8)
    d = int(n * 0.4)
    return q, g, d


# ABLATION VARIANTS
def aqir_ads_full(img, depth):
    gx = cv2.Sobel(img, cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(img, cv2.CV_32F, 0, 1)
    grad = np.sqrt(gx**2 + gy**2)

    theta = (img / 255.0) * (math.pi / 2)
    theta *= (1 + 0.15 * grad / (grad.max() + 1e-6))

    recon = np.uint8((theta / (math.pi / 2)) * 255)
    noise = np.random.normal(0, 255 * (depth / 200), img.shape)

    return np.clip(recon + noise, 0, 255).astype(np.uint8)


def aqir_no_grad(img, depth):
    theta = (img / 255.0) * (math.pi / 2)
    recon = np.uint8((theta / (math.pi / 2)) * 255)
    noise = np.random.normal(0, 255 * (depth / 200), img.shape)
    return np.clip(recon + noise, 0, 255).astype(np.uint8)


def aqir_fixed_rotation(img, depth):
    theta = np.full(img.shape, math.pi / 4)
    recon = np.uint8((theta / (math.pi / 2)) * 255)
    noise = np.random.normal(0, 255 * (depth / 200), img.shape)
    return np.clip(recon + noise, 0, 255).astype(np.uint8)


def aqir_deep(img, depth):
    increased_depth = int(depth * 2)
    return aqir_ads_full(img, increased_depth)


ABLATION_VARIANTS = {
    "AQIR-ADS": aqir_ads_full,
    "AQIR-NoGrad": aqir_no_grad,
    "AQIR-FixedRot": aqir_fixed_rotation,
    "AQIR-Deep": aqir_deep
}


# RUN ABLATION
records = []

for dataset in DATASETS:
    dataset_path = os.path.join(BASE_PATH, dataset)
    files = os.listdir(dataset_path)[:MAX_IMAGES]

    for img_name in files:
        img = load_image(os.path.join(dataset_path, img_name))
        if img is None or img.size == 0:
            continue

        for res, dim in RESIZE_DIMS.items():
            resized = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
            n_pixels = resized.size
            _, _, depth = aqir_ads(n_pixels)

            for variant, func in ABLATION_VARIANTS.items():
                noisy = func(resized, depth)

                records.append({
                    "Variant": variant,
                    "Dataset": dataset,
                    "Resolution": res,
                    "PSNR": compute_psnr(resized, noisy),
                    "SSIM": compute_ssim(resized, noisy),
                    "GateFidelity": gate_fidelity(depth),
                    "InformationLoss": round(1 - compute_ssim(resized, noisy), 3)
                })

# Save
ablation_df = pd.DataFrame(records)
ablation_df.to_csv("Table_Ablation_AQIR_ADS.csv", index=False)

print("Ablation study completed successfully.")
print(ablation_df.groupby("Variant").mean(numeric_only=True))

Ablation study completed successfully.
                    PSNR      SSIM  GateFidelity  InformationLoss
Variant                                                          
AQIR-ADS       41.081735  0.904917        0.9335         0.095083
AQIR-Deep      34.498012  0.789927        0.9335         0.210073
AQIR-FixedRot   7.484833  0.136018        0.9335         0.863982
AQIR-NoGrad    40.360602  0.904640        0.9335         0.095360


In [27]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

# SPRINGER CONFIGURATION
SPRINGER_IMG_DIR = "SPRINGER_READY_IMAGES"
os.makedirs(SPRINGER_IMG_DIR, exist_ok=True)

DPI = 600   # Springer standard
FIG_SIZE = (15, 5)


# LOAD BRAIN TUMOR IMAGE

DATASET_PATH = r"C:\Users\Aditi\comparison_qir\input_datasets\braintumor"
image_name = sorted(os.listdir(DATASET_PATH))[0]
image_path = os.path.join(DATASET_PATH, image_name)

img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
if img is None:
    raise ValueError("Failed to load image")

# Normalize to quantum amplitude range [0,1]
I = img.astype(np.float32) / 255.0


# (1) PAULI-X GATE (QUANTUM FLIP)

# |ψ> → X|ψ>
# Pixel-wise inversion
X_gate = 1.0 - I


# (2) ROTATION GATE Ry(θ)

theta = I * np.pi / 2
Ry_gate = np.sin(theta) ** 2


# (3) PHASE ENCODING (GRADIENT-WEIGHTED)

gx = cv2.Sobel(I, cv2.CV_32F, 1, 0, ksize=3)
gy = cv2.Sobel(I, cv2.CV_32F, 0, 1, ksize=3)

gradient = np.sqrt(gx**2 + gy**2)
grad_norm = gradient / (np.max(gradient) + 1e-6)

phi = 0.1 * np.pi + grad_norm * (0.5 * np.pi)

# Phase-modulated state |ψ'> = |ψ| * cos(φ)
Phase_gate = I * np.cos(phi)

# VISUALIZATION (SPRINGER STYLE)
fig, axes = plt.subplots(1, 4, figsize=FIG_SIZE)

axes[0].imshow(I, cmap="gray")
axes[0].set_title("(a) Input Image")
axes[0].axis("off")

axes[1].imshow(X_gate, cmap="gray")
axes[1].set_title("(b) Pauli-X (Flip)")
axes[1].axis("off")

axes[2].imshow(Ry_gate, cmap="gray")
axes[2].set_title("(c) Rotation $R_y(\\theta)$")
axes[2].axis("off")

axes[3].imshow(Phase_gate, cmap="gray")
axes[3].set_title("(d) Phase Encoding")
axes[3].axis("off")

plt.tight_layout()

# SAVE SPRINGER-READY IMAGE
save_path = os.path.join(
    SPRINGER_IMG_DIR,
    "basic_operations.png"
)

plt.savefig(save_path, dpi=DPI, bbox_inches="tight")

print("Springer-ready image saved at:", save_path)

Springer-ready image saved at: SPRINGER_READY_IMAGES\basic_operations.png
